In [2]:
import sys
import os
from pathlib import Path

# Add the project root to Python path so we can import src
# Get current working directory
cwd = Path().resolve()

# If we're in the notebooks directory, go up one level
# Otherwise, assume we're already in the project root
if cwd.name == "notebooks":
	project_root = cwd.parent
else:
	project_root = cwd

if str(project_root) not in sys.path:
	sys.path.insert(0, str(project_root))


In [3]:
from langchain.agents import create_agent
from langchain.tools import tool
from src.rag.retrieval.index import retrieve_context
from src.rag.retrieval.utils import prepare_prompt_and_invoke_llm
from src.config.index import appConfig
import json

from langgraph.graph import MessagesState
from typing import Any, List, Dict
from typing_extensions import Annotated
from langgraph.types import Command
from langchain_core.tools.base import InjectedToolCallId
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableConfig

In [10]:
# Create a custom agent state that extends the MessagesState to store citations
class CustomAgentState(MessagesState):
	"""Extended agent state with citations tracking"""
	# citations will accumulate across tool calls
	citations: Annotated[List[Dict[str, Any]], lambda x, y: x + y] = []

In [11]:
# Factory function to create RAG tool with project_id bound
def create_rag_tool(project_id: str):
	"""Create a RAG search tool bound to a specific project"""
	
	@tool
	def rag_search(
		query: str,
		tool_call_id: Annotated[str, InjectedToolCallId],
	) -> Command:
		"""
		Search through project documents using RAG (Retrieval-Augmented Generation).
		This tool retrieves relevant context from the current project's documents based on the query.
		
		Args:
			query: The search query or question to find relevant information
			
		Returns:
			A formatted string containing the retrieved context and answer based on the documents
		"""
		try:
			# Retrieve context using the existing RAG pipeline
			texts, images, tables, citations = retrieve_context(project_id, query)
			
			# If no context found, return a message
			if not texts and not images and not tables:
				return Command(
					update={
						"messages": [
							ToolMessage(
								"No relevant information found in the project documents for this query.",
								tool_call_id=tool_call_id
							)
						]
					}
				)
				
			# Prepare the response using the existing LLM preparation function
			response = prepare_prompt_and_invoke_llm(
				user_query=query,
				texts=texts,
				images=images,
				tables=tables
			)
			
			return Command(
                update={
                    # Update message history
                    "messages": [
                        ToolMessage(
                            content=response,
                            tool_call_id=tool_call_id
                        )
                    ],
                    # Update citations in state - these accumulate!
                    "citations": citations
                }
            )		
		except Exception as e:
			return Command(update={
                    "messages": [
                        ToolMessage(
                            f"Error retrieving information: {str(e)}",
                            tool_call_id=tool_call_id
                        )
                    ]
                }
            )

	return rag_search

In [12]:
# Create agent with project-specific RAG tool
def create_rag_agent(project_id: str, model: str = "gpt-4o"):
	"""Create an agent with RAG tool for a specific project"""
	
	# Create tools list with project-specific RAG tool
	tools = [create_rag_tool(project_id)]
	
	# Define the system prompt
	system_prompt = """You are a helpful AI assistant with access to a RAG (Retrieval-Augmented Generation) tool that searches project-specific documents.

For every user question:

1. Do not assume any question is purely conceptual or general.  
2. Use the `rag_search` tool immediately with a clear and relevant query derived from the user’s question.  
3. Carefully review the retrieved documents and base your entire answer on the RAG results.  
4. If the retrieved information fully answers the user’s question, respond clearly and completely using that information.  
5. If the retrieved information is insufficient or incomplete, explicitly state that and provide helpful suggestions or guidance based on what you found.  
6. Always present answers in a clear, well-structured, and conversational manner.

**Never answer without first querying the RAG tool. This ensures every response is grounded in project-specific context and documentation.**
"""
	
	# Create the agent graph
	agent = create_agent(
		model=model,
		tools=tools,
		system_prompt=system_prompt,
		state_schema=CustomAgentState
	)
	
	return agent

In [16]:
project_id = "6d090d75-7c7c-428c-bba8-258cf3f45d2d"
rag_agent = create_rag_agent(project_id=project_id, model="gpt-4o")

In [ ]:
inputs = {"messages": [{"role": "user", "content": "What are the two types of sleep?"}]}

result = rag_agent.invoke(inputs)


Vector search resulted in: 3 chunks
🤖 Invoking LLM with 2 messages (3 texts, 0 tables, 0 images)...
{'messages': [HumanMessage(content='What are the two types of sleep?', additional_kwargs={}, response_metadata={}, id='86c8d3b8-bc56-4d80-9e20-93137190115e'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 286, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ed643dde95', 'id': 'chatcmpl-Cef9KlffZxqobXBI3WI0LHLAY5TWn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4b045f9f-37cb-4978-915f-30d11beb30e6-0', tool_calls=[{'name': 'rag_search', 'args': {'query': 'types of sleep'}, 'id': 'call_EYzUAlv6Z

In [19]:
result

{'messages': [HumanMessage(content='What are the two types of sleep?', additional_kwargs={}, response_metadata={}, id='86c8d3b8-bc56-4d80-9e20-93137190115e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 286, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ed643dde95', 'id': 'chatcmpl-Cef9KlffZxqobXBI3WI0LHLAY5TWn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4b045f9f-37cb-4978-915f-30d11beb30e6-0', tool_calls=[{'name': 'rag_search', 'args': {'query': 'types of sleep'}, 'id': 'call_EYzUAlv6ZllaZXwNOBnB88m6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 286, 'output_tokens': 17,